In [9]:
from model import GPTConfig, GPT
import torch

ckpt_file = "../models/random_karvhypNSNR_finetune/random_karvhypNSNR_finetune_0K.pth"
n_layer = 8
n_head = 8
n_embd = 512
block_size = 1023
bias = False
dropout = 0.0


model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line
checkpoint = torch.load(ckpt_file, map_location = 'cpu')

print(checkpoint.keys())
print(checkpoint["iter_num"])
#checkpoint["iter_num"] = 0

#print(checkpoint["iter_num"])
#torch.save(checkpoint, ckpt_file)
#checkpoint_model_args = checkpoint['model_args']
## force these config attributes to be equal otherwise we can't even resume training
## the rest of the attributes (e.g. dropout) can stay as desired from command line
#for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
    #model_args[k] = checkpoint_model_args[k]
## create the model


dict_keys(['model', 'optimizer', 'model_args', 'iter_num', 'train_loss_list', 'val_loss_list', 'best_val_loss', 'config'])
0


/tmp/ipykernel_29008/1669191510.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_file, map_location = 'cpu')


In [ ]:
gptconf = GPTConfigMod(**model_args)
model = GPT(gptconf)
state_dict = checkpoint['model']
# fix the keys of the state dictionary :(
# honestly no idea how checkpoints sometimes get this prefix, have to debug more
unwanted_prefix = '_orig_mod.'
iter_num = checkpoint['iter_num']
try:
    train_loss_list = checkpoint['train_loss_list']
except Exception as e:
    print('no train loss list found defaulting to empty')
try:
    val_loss_list = checkpoint['val_loss_list']
except Exception as e:
    print('no val loss list found defaulting to empty')
flag = False
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        flag = True
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

/tmp/ipykernel_4322/2104534906.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_file, map_location = 'cpu')


number of parameters: 25.19M


<All keys matched successfully>

In [ ]:
from transformers import PretrainedConfig

class GPTConfigMod(PretrainedConfig):
    model_type = "gpt"

    def __init__(
        self,
        block_size=1024,
        vocab_size=32,
        n_layer=8,
        n_head=8,
        n_embd=512,
        dropout=0.0,
        bias=True,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout
        self.bias = bias


In [3]:
from peft import LoraConfig, get_peft_model

# LoRA configuration
lora_config = LoraConfig(
    r=8,  # Low-rank dimension
    lora_alpha=16,  # Scaling factor
    target_modules=[
        "attn.c_attn",  # Query, Key, Value
        "attn.c_proj",  # Output projection
        "mlp.c_fc",     # Feed-forward first layer
        "mlp.c_proj"    # Feed-forward projection layer
    ],
    lora_dropout=0.1,  # Dropout rate
    bias="none",  # Bias handling
)

# Apply LoRA to the model

peft_model = get_peft_model(model, lora_config)

AttributeError: 'GPTConfig' object has no attribute 'get'

In [3]:
from transformers import GPT2Config, GPT2LMHeadModel

# Define a custom GPT-2 configuration with 512 hidden size, 8 heads, and 8 layers
config = GPT2Config(
    n_embd=512,          # Hidden size
    n_layer=8,           # Number of layers
    n_head=8,            # Number of attention heads
    vocab_size=32,    # Example vocab size
    max_position_embeddings=1023, # Max position embeddings
    n_inner=2048         # Feed-forward layer size
)

# Create the model with the custom configuration
model = GPT2LMHeadModel(config)

In [7]:
from transformers import AutoModelForSeq2SeqLM
model_path = "evaluation/eval_models/lichess9gb_8layer_23K.pth"
model = AutoModelForSeq2SeqLM.from_pretrained("MarcosElFlamenco/lichess9gb_8layer_23K.pth")

ValueError: Unrecognized model in MarcosElFlamenco/lichess9gb_8layer_23K.pth. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, audio-spectrogram-transformer, autoformer, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, deta, detr, dinat, dinov2, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, encodec, encoder-decoder, ernie, ernie_m, esm, falcon, falcon_mamba, fastspeech2_conformer, flaubert, flava, fnet, focalnet, f

In [6]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

AttributeError: 'dict' object has no attribute '__dict__'